
# ETL Process: Data Extraction, Transformation, and Loading

This notebook implements an **ETL (Extract, Transform, Load) pipeline**, which is commonly used for processing structured datasets. The key objectives of this notebook are:

1. **Data Extraction**: Loading raw data from various sources (CSV, databases, APIs).
2. **Data Transformation**: Cleaning, normalizing, and restructuring the data.
3. **Data Loading**: Storing the processed data in a suitable format (database, CSV, or other storage).

## Summary of Assets:

- **Input Data**: Raw datasets extracted from different sources.
- **Transformation Steps**: Data cleaning, normalization, type conversion, handling missing values.
- **Output Data**: Processed and structured data ready for analysis or storage.

Below, the code is interleaved with explanatory Markdown sections to clarify each step.


**Processing Step**: This section performs data operations as part of the ETL pipeline.

In [ ]:
# # Script tonto para eliminar caracteres sobrantes del 
# # archivo csv (Draft_AnonymousAttacks.csv)

import csv
import re

def clean_csv(input_path, output_path):
    with open(input_path, 'r', newline='', encoding='utf-8') as infile, \
         open(output_path, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        for row in reader:
#             # Eliminar todos los corchetes con números dentro en cada columna
            cleaned_row = [re.sub(r'\s*\[\d+\]\s*', '', cell) for cell in row]
            writer.writerow(cleaned_row)

def main():
    input_path = input("Ingrese la ruta del archivo de entrada: ")
    output_folder = input("Ingrese la ruta de la carpeta de salida: ")
    output_filename = input("Ingrese el nombre del archivo de salida (sin extensión): ")
    
    output_path = f"{output_folder}/{output_filename}.csv"

    clean_csv(input_path, output_path)
    print(f"Archivo limpio guardado en: {output_path}")

if __name__ == "__main__":
    main()

**Library Imports**: This section loads essential Python libraries for data manipulation and ETL tasks.

In [ ]:
import csv
import os

def leer_diccionario(ruta_diccionario):
    with open(ruta_diccionario, 'r', encoding='utf-8') as f:
        return [linea.strip() for linea in f if linea.strip()]

def buscar_coincidencias(fila, diccionario):
    return '; '.join(palabra for palabra in diccionario if palabra.lower() in fila.lower())

def procesar_csv(ruta_entrada, ruta_salida, diccionario, nombre_columna):
    with open(ruta_entrada, 'r', newline='', encoding='utf-8') as archivo_entrada, \
         open(ruta_salida, 'w', newline='', encoding='utf-8') as archivo_salida:
        
        lector = csv.reader(archivo_entrada, delimiter=';')
        escritor = csv.writer(archivo_salida, delimiter=';')
        
        encabezados = next(lector)
        encabezados.append(nombre_columna)
        escritor.writerow(encabezados)
        
        for fila in lector:
            coincidencias = buscar_coincidencias(';'.join(fila), diccionario)
            fila.append(coincidencias)
            escritor.writerow(fila)

def main():
    ruta_diccionario = input("Introduce la ruta del archivo de diccionario (.txt): ")
    ruta_csv_entrada = input("Introduce la ruta del archivo CSV de entrada: ")
    ruta_salida = input("Introduce la ruta para el archivo de salida: ")
    nombre_columna = input("Introduce el nombre para la nueva columna: ")
    nombre_archivo_salida = input("Introduce el nombre para el archivo de salida (sin .csv): ")
    
    ruta_csv_salida = os.path.join(ruta_salida, f"{nombre_archivo_salida}.csv")
    
    diccionario = leer_diccionario(ruta_diccionario)
    procesar_csv(ruta_csv_entrada, ruta_csv_salida, diccionario, nombre_columna)
    
    print(f"Proceso completado. El archivo de salida se encuentra en: {ruta_csv_salida}")

if __name__ == "__main__":
    main()

**Processing Step**: This section performs data operations as part of the ETL pipeline.

In [ ]:
# # Web Scraping (wikipedia) para obtener un csv con
# # los datos necesitados del grupo Anonymous


import csv
import requests
from bs4 import BeautifulSoup

# # Pedir la URL y el nombre del archivo CSV
url_html = input("Ingresa la URL del archivo HTML: ")
ruta_csv = input("Ingresa la ruta donde se guardará el archivo CSV: ")
nombre_csv = input("Ingresa el nombre del archivo CSV (sin extensión): ")

# # Realizar la petición HTTP para obtener el contenido del HTML
response = requests.get(url_html)
response.raise_for_status()  # Levanta un error si la petición no es exitosa

# # Analizar el contenido HTML con BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# # Crear una lista para almacenar los datos
data = []

# # Variables para almacenar el año y el título actuales
current_year = ""
current_title = ""

# # Buscar todos los elementos <div> con clase mw-heading
sections = soup.find_all('div', class_='mw-heading')

for section in sections:
    if 'mw-heading2' in section['class']:
#         # Es un nuevo año
        current_year = section.get_text(strip=True)
        current_title = ""  # Resetear el título para el caso donde no haya
        sibling = section.find_next_sibling()
        
#         # Manejar casos antes de 2011 donde el cuerpo del incidente está dentro de <li>
        if sibling and sibling.name == 'ul':
            for li in sibling.find_all('li'):
                description = li.get_text(strip=True)
                data.append([current_year, "Sin título", description])
                
    elif 'mw-heading3' in section['class']:
#         # Es un nuevo incidente con título
        current_title = section.get_text(strip=True)
        description = ""
        sibling = section.find_next_sibling()
        
#         # Recorrer los hermanos siguientes para construir la descripción
        while sibling and not any('mw-heading' in cls for cls in sibling.get('class', [])):
            if sibling.name == 'p':
#                 # Manipular <a href> dentro de <p> para añadir espacios antes y después
                for a in sibling.find_all('a'):
                    a.insert_before(' ')
                    a.insert_after(' ')
                description += sibling.get_text() + " "  # Eliminar strip() para preservar los espacios
            sibling = sibling.find_next_sibling()

#         # Limpiar y agregar los datos a la lista
        description = description.strip()
        if current_year and description:
            data.append([current_year, current_title, description])

# # Guardar los datos en un archivo CSV usando punto y coma como delimitador
csv_file = f"{ruta_csv}/{nombre_csv}.csv"
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writerow(['Año', 'Título', 'Descripción'])
    writer.writerows(data)

print(f"Archivo CSV guardado en: {csv_file}")
